In [1]:
from network import *

In [2]:
# 读取所有设备的IQ数据
device_files = {
    "bladerf": "../raw data/bladerf/bladerf.iq",
    "hackrf0": "../raw data/hackrf0/hackrf0.iq",
    "hackrf1": "../raw data/hackrf1/hackrf1.iq",
    "limesdr": "../raw data/limesdr/limesdr.iq"
}

all_data = []
all_labels = []

for label_idx, (device_name, file_path) in enumerate(device_files.items()):
    print(f"正在处理 {device_name} 的数据...")
    # 读取数据
    data = load_iq_file(file_path)
    
    # 分段数据（将连续IQ数据分成固定长度的片段）
    segment_length = 1024  # 每个样本的IQ点数，根据需要调整
    num_segments = len(data) // segment_length
    segments = data[:num_segments * segment_length].reshape(num_segments, segment_length)
    
    # 添加到数据集
    all_data.append(segments)
    all_labels.append(np.full(num_segments, label_idx))
    
    print(f"处理完成，获取了 {num_segments} 个样本")

正在处理 bladerf 的数据...
处理完成，获取了 48828 个样本
正在处理 hackrf0 的数据...
处理完成，获取了 48828 个样本
正在处理 hackrf1 的数据...
处理完成，获取了 48828 个样本
正在处理 limesdr 的数据...
处理完成，获取了 48828 个样本


In [3]:
# 合并数据
X = np.vstack(all_data)
y = np.concatenate(all_labels)

# 分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"训练集样本数: {len(X_train)}")
print(f"测试集样本数: {len(X_test)}")

训练集样本数: 156249
测试集样本数: 39063


In [4]:
# 创建数据加载器
batch_size = 32
train_loader, test_loader, scalers = create_dataloaders(
    X_train, y_train, X_test, y_test, batch_size)

In [5]:
# 初始化模型
num_features = X_train.shape[1]  # 每个样本的IQ点数
num_classes = 4  # bladerf, hackrf0, hackrf1, limesdr
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model(num_features, num_classes).to(device)

In [6]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 训练循环
num_epochs = 30
for epoch in range(num_epochs):
    # 训练
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        # print(inputs,labels)
    
    print(f'第 {epoch+1}/{num_epochs} 轮, 损失: {train_loss/len(train_loader):.4f}, 准确率: {100.*correct/total:.2f}%')
    
    # 验证
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    print(f'测试损失: {test_loss/len(test_loader):.4f}, 测试准确率: {100.*correct/total:.2f}%')


第 1/30 轮, 损失: 1.3174, 准确率: 33.42%
测试损失: 0.9980, 测试准确率: 49.71%
第 2/30 轮, 损失: 0.6055, 准确率: 69.07%
测试损失: 0.2479, 测试准确率: 91.06%
第 3/30 轮, 损失: 0.2637, 准确率: 87.75%
测试损失: 0.2389, 测试准确率: 88.53%
第 4/30 轮, 损失: 0.2003, 准确率: 91.28%
测试损失: 0.3918, 测试准确率: 82.47%
第 5/30 轮, 损失: 0.1550, 准确率: 93.49%
测试损失: 0.2749, 测试准确率: 87.56%
第 6/30 轮, 损失: 0.1309, 准确率: 94.60%
测试损失: 0.1202, 测试准确率: 95.07%
第 7/30 轮, 损失: 0.1152, 准确率: 95.32%
测试损失: 0.0756, 测试准确率: 97.04%
第 8/30 轮, 损失: 0.1040, 准确率: 95.80%
测试损失: 0.1001, 测试准确率: 95.88%
第 9/30 轮, 损失: 0.0959, 准确率: 96.19%
测试损失: 0.0630, 测试准确率: 97.60%
第 10/30 轮, 损失: 0.0873, 准确率: 96.53%
测试损失: 0.0721, 测试准确率: 97.25%
第 11/30 轮, 损失: 0.0824, 准确率: 96.73%
测试损失: 0.0661, 测试准确率: 97.49%
第 12/30 轮, 损失: 0.0782, 准确率: 96.95%
测试损失: 0.0637, 测试准确率: 97.48%
第 13/30 轮, 损失: 0.0751, 准确率: 97.08%
测试损失: 0.0856, 测试准确率: 96.53%
第 14/30 轮, 损失: 0.0699, 准确率: 97.29%
测试损失: 0.0684, 测试准确率: 97.29%
第 15/30 轮, 损失: 0.0672, 准确率: 97.37%
测试损失: 0.0439, 测试准确率: 98.36%
第 16/30 轮, 损失: 0.0632, 准确率: 97.54%
测试损失: 0.0491, 测试准确率: 98.04%
第

In [15]:
# 保存模型
# torch.save(model.state_dict(), './models/complex_cnn_model.pth')

torch.save(scalers, './models/scalers.pth')

In [13]:
def predict_device(model, iq_file_path, scalers, device_names, confidence_threshold=0.7):
    """预测设备类型，包含未知设备检测"""
    # 读取并处理数据
    data = load_iq_file(iq_file_path)
    segment_length = 1024
    num_segments = len(data) // segment_length
    segments = data[:num_segments * segment_length].reshape(num_segments, segment_length)
    
    # 创建数据集和加载器
    dataset = ComplexSignalDataset(segments, np.zeros(len(segments)), scalers[0], scalers[1])
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    
    # 预测
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = F.softmax(outputs, dim=1)
            all_probs.append(probs.cpu().numpy())
    
    all_probs = np.vstack(all_probs)
    
    # 获取最高概率及其对应的类别
    max_probs = np.max(all_probs, axis=1)
    predictions = np.argmax(all_probs, axis=1)
    
    # 计算每个段的平均置信度
    avg_confidence = np.mean(max_probs)
    
    # 统计各类别的数量
    unique, counts = np.unique(predictions, return_counts=True)
    result = dict(zip(unique, counts))
    final_result = {device_names[int(k)]: v for k, v in result.items()}
    
    # 判断是否为未知设备
    if avg_confidence < confidence_threshold:
        return "未知设备", final_result, avg_confidence
    else:
        # 返回出现最多的设备类型
        majority_device = max(final_result, key=final_result.get)
        return majority_device, final_result, avg_confidence

In [ ]:
# 加载模型和标准化器
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = 1024  # 与训练时相同
num_classes = 4  # 四种已知设备
model = create_model(num_features, num_classes).to(device)
model.load_state_dict(torch.load('complex_cnn_model.pth'))
scalers = torch.load('scalers.pth')

# 使用模型预测未知设备
unknown_file = "path/to/unknown_device.iq"
device_type, class_counts, confidence = predict_device(
    model, unknown_file, scalers, device, confidence_threshold=0.7)

print(f"预测的设备类型: {device_type}")
print(f"各类别的预测数量: {class_counts}")
print(f"平均置信度: {confidence:.4f}")

# 如果是已知设备，可以进一步分析
if device_type != "未知设备":
    print(f"这是一个已知设备: {device_type}")
else:
    print("这可能是一个训练数据中未包含的设备！")